In [1]:
# Import libraries

import sys
import pypsa
import logging
import warnings
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
from plotly import express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import ipywidgets as widgets
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.demand import projected_energy
from library.weather import generate_cutout
from library.renewables import capacity_factor

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = False
use_h2 = True
h2_initial = 1000
biogas_limit = 0
load_target = 15

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
# target_load = projected_energy(target_year, 1.21265) * normalized_demand['se3'].values.flatten() * 1_000_000
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
cutout, selections, eez, index = generate_cutout(geo, None, '2023-01', '2023-12')
selection = selections[geo]

capacity_factor_solar = capacity_factor(cutout, selection, 'solar', '', geo, None, '2023-01', '2023-12').values.flatten()
capacity_factor_onwind = capacity_factor(cutout, selection, 'onwind', onwind_turbine, geo, None, '2023-01', '2023-12').values.flatten()
capacity_factor_offwind = capacity_factor(cutout, selection, 'offwind', offwind_turbine, geo, None, '2023-01', '2023-12').values.flatten()


/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/cutout.py:191: UserWarning:

Arguments module, x, y, time, dx, dy, dt are ignored, since cutout is already built.

INFO:atlite.data:Storing temporary files in /tmp/tmpkx3qqw80
INFO:atlite.convert:Convert and aggregate 'pv'.


[########################################] | 100% Completed | 4.62 s


/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/resource.py:72: FutureWarning:

'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.

INFO:atlite.convert:Convert and aggregate 'wind'.
/home/viktor/miniforge3/envs/pypsa-vgr/lib/python3.11/site-packages/atlite/resource.py:72: FutureWarning:

'add_cutout_windspeed' for wind turbine
power curves will default to True in atlite relase v0.2.13.

INFO:atlite.convert:Convert and aggregate 'wind'.


In [4]:
# Build the network

def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

def annualized_capex(asset):
    return (annuity(float(assumptions.loc[('general', 'discount_rate'), 'value']), float(assumptions.loc[(asset, 'lifetime'), 'value'])) + float(assumptions.loc[(asset, 'FOM'), 'value'])) * float(assumptions.loc[(asset, 'capital_cost'), 'value'])

## Initialize the network
network = pypsa.Network()
network.set_snapshots(index)
network.snapshot_weightings.loc[:, :] = resolution

## Carriers
carriers = [
    'AC',
    'onwind',
    'offwind',
    'solar',
    'li-ion',
    'h2',
    'biogas',
    'mixedgas',
    'backstop',
    ]

carrier_colors = ['black', 'green', 'blue', 'red', 'lightblue', 'grey', 'brown', 'brown', 'white']

network.madd(
    'Carrier',
    carriers,
    color=carrier_colors,
    )

## Load bus location
minx, miny, maxx, maxy = selection.total_bounds
midx = (minx + maxx)/2
midy = (miny + maxy)/2

## Add the buses
network.add('Bus', 'load-bus', carrier='AC', x=midx, y=midy)
network.add('Bus', 'renewables-bus', x=midx+0.5, y=midy+0.25)
network.add('Bus', 'battery-bus', carrier='li-ion', x=midx-0.5, y=midy)
if use_h2 or biogas_limit > 0:
    network.add('Bus', 'turbine-bus', x=midx, y=midy+0.5)
if use_h2:
    network.add('Bus', 'h2-bus', carrier='h2', x=midx-0.5, y=midy+0.5)
if biogas_limit > 0:
    network.add('Bus', 'biogas-bus', x=midx, y=midy+0.9)


## Add load and backstop to load bus
network.add('Load', 'load', bus='load-bus',
            p_set=target_load
            )

network.add('Generator', 'backstop', carrier='backstop', bus='load-bus',
            p_nom_extendable=True,
            capital_cost=assumptions.loc[('backstop', 'capital_cost'), 'value'],
            marginal_cost=assumptions.loc[('backstop', 'marginal_cost'), 'value'],
            lifetime=assumptions.loc[('backstop', 'lifetime'), 'value'],
            )

## Add generators and links to renewable bus

network.add('Generator', 'solar', carrier='solar', bus='renewables-bus',
            p_nom_extendable=True, 
            p_max_pu=capacity_factor_solar,
            p_nom_mod=assumptions.loc['solar','unit_size'].value,
            capital_cost= annualized_capex('solar'),
            marginal_cost=assumptions.loc[('solar', 'VOM'), 'value'],
            lifetime=assumptions.loc[('solar', 'lifetime'), 'value'],
            )

network.add('Generator', 'onwind', carrier='onwind', bus='renewables-bus',
            p_nom_extendable=True,
            p_max_pu=capacity_factor_onwind,
            p_nom_mod=assumptions.loc['onwind','unit_size'].value,
            capital_cost= annualized_capex('onwind'),
            marginal_cost=assumptions.loc[('onwind', 'VOM'), 'value'],
            lifetime=assumptions.loc['onwind','lifetime'].value,
            )

if use_offwind:
    network.add('Generator', 'offwind', carrier='offwind', bus='renewables-bus',
                p_nom_extendable=use_offwind,
                p_max_pu=(capacity_factor_offwind if use_offwind else [0] * len(capacity_factor_offwind)),
                p_nom_mod=assumptions.loc['offwind','unit_size'].value,
                capital_cost= annualized_capex('offwind'),
                marginal_cost=assumptions.loc[('offwind', 'VOM'), 'value'],
                lifetime=assumptions.loc['offwind','lifetime'].value,
                )

network.add('Link', 'Renewables load link', bus0='renewables-bus', bus1='load-bus',
            p_nom_extendable=use_offwind,
            )

## Add battery storage

network.add('Link','battery-charge', bus0='renewables-bus', bus1='battery-bus',
            p_nom_extendable = True,
            capital_cost= annualized_capex('battery_inverter'),
            marginal_cost=assumptions.loc['battery_inverter','VOM'].value,
            lifetime=assumptions.loc['battery_inverter','lifetime'].value,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

network.add('Store', 'battery', carrier='li-ion', bus='battery-bus',
            e_initial=100,
            e_nom_extendable=True,
            e_cyclic=True,
            e_min_pu=0.15,
            standing_loss=0.00008, # TODO: Check if this is really per hour as in the documentation or if it is per snapshot
            capital_cost= annualized_capex('battery_storage'),
            marginal_cost=assumptions.loc['battery_storage','VOM'].value,
            lifetime=assumptions.loc['battery_storage', 'lifetime'].value,
            )

network.add('Link','battery-discharge', carrier='li-ion', bus0='battery-bus', bus1='load-bus',
            p_nom_extendable = True,
            efficiency = assumptions.loc['battery_inverter','efficiency'].value,
            )

## Add H2 electrolysis, storage, pipline to gas turbine

if use_h2:
    network.add('Link', 'h2-electrolysis', carrier='h2', bus0='renewables-bus', bus1='h2-bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['h2_electrolysis','unit_size'].value,
                capital_cost= annualized_capex('h2_electrolysis'),
                marginal_cost=assumptions.loc[('h2_electrolysis', 'VOM'), 'value'],
                lifetime=assumptions.loc['h2_electrolysis','lifetime'].value,
                efficiency=assumptions.loc['h2_electrolysis','efficiency'].value,
                )

    network.add('Store', 'h2', carrier='h2', bus='h2-bus',
                e_initial=(150_000 if use_h2 else 0),
                e_nom_extendable=use_h2,
                e_cyclic=True,
                capital_cost= annualized_capex('h2_storage'),
                marginal_cost=assumptions.loc['h2_storage','VOM'].value,
                lifetime=assumptions.loc['h2_storage','lifetime'].value
                )

    network.add('Link', 'H2 pipeline', carrier='h2', bus0='h2-bus', bus1='turbine-bus',
                p_nom_extendable=True,
                )

### Biogas pipeline

if biogas_limit > 0:
    network.add('Generator', 'biogas-market', carrier='biogas', bus='biogas-bus',
                p_nom_extendable=True,
                p_nom_max=biogas_limit,
                marginal_cost=assumptions.loc['biogas','cost'].value,
                lifetime=100,
                )

    network.add('Link', 'Biogas pipeline', carrier='biogas', bus0='biogas-bus', bus1='turbine-bus',
                p_nom_extendable=True,
                )

### Gas turbines
if use_h2 or biogas_limit > 0:
         
    network.add('Link', 'gas-turbine', carrier='mixedgas', bus0='turbine-bus', bus1='load-bus',
                p_nom_extendable=True,
                p_nom_mod=assumptions.loc['combined_cycle_gas_turbine','unit_size'].value,
                capital_cost= annualized_capex('combined_cycle_gas_turbine'),
                marginal_cost=assumptions.loc['combined_cycle_gas_turbine','VOM'].value,
                lifetime=assumptions.loc['combined_cycle_gas_turbine','lifetime'].value,
                efficiency=assumptions.loc['combined_cycle_gas_turbine','efficiency'].value,
                )

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

generator_capacity = model.variables["Generator-p_nom"]
link_capacity = model.variables["Link-p_nom"]

## Add offwind constraint
if use_offwind:
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 292.28it/s]
INFO:linopy.io: Writing time: 1.35s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 1e+06]
  Bound  [0e+00, 0e+00]
  RHS    [1e+03, 3e+03]
Presolving model
42346 rows, 30673 cols, 103678 nonzeros  0s
34380 rows, 22707 cols, 91758 nonzeros  0s
33655 rows, 21982 cols, 94152 nonzeros  0s

Solving MIP model with:
   33655 rows
   21982 cols (0 binary, 4 integer, 0 implied int., 21978 continuous)
   94152 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.5s
 S       0       0         0   0.00%   -inf            26107928839.92     Large        0      0      0         0    16.7s
 R       0       0         0  

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 37974 primals, 84695 duals
Objective: 2.61e+10
Solver model: available
Solver message: optimal



s

Solving report
  Status            Optimal
  Primal bound      26102293072.7
  Dual bound        26102266894.4
  Gap               0.0001% (tolerance: 0.01%)
  Solution status   feasible
                    26102293072.7 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            17.49 (total)
                    0.41 (presolve)
                    0.00 (postsolve)
  Nodes             1
  LP iterations     27014 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)
Writing the solution to /tmp/linopy-solve-smqywhkd.sol


INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-fix-p-lower, Link-fix-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop      2.529574e+03                 0.0  6.682682e+05   
          onwind        6.997397e+03                 0.0  8.215390e+06   
          solar         7.235991e+03                 0.0  7.369379e+06   
Link      AC            3.934486e+03                 0.0  1.494838e+07   
          h2            4.221517e-12                 0.0  4.394303e-10   
          li-ion        4.101987e+03                 0.0  1.433173e+07   
          mixedgas      0.000000e+00                 0.0  2.423830e-10   
Load      -             0.000000e+00                 0.0  0.000000e+00   
Store     h2            1.813069e-12                 0.0  1.865088e-10   
          li-ion        2.497945e+04                 0.0  3.178222e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  6.682682e+05  0.000000e+00         0.030158   
          onwind    0.000000e+00  8.215390e+06  0.000000e+00         0.134026   
          solar     0.000000e+00  7.369379e+06  0.000000e+00         0.116260   
Link      AC        1.558477e+07 -6.363884e+05  0.000000e+00         0.452177   
          h2        3.803391e-10  5.909128e-11  0.000000e+00         0.012127   
          li-ion    1.494187e+07 -6.101362e+05  0.000000e+00         0.415821   
          mixedgas  1.848374e-10  5.754561e-11 -1.350836e-10              NaN   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     h2        2.281265e-10 -4.161774e-11  0.000000e+00         0.631372   
          li-ion    3.184734e+06 -6.511916e+03  0.000000e+00         0.372020   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          onwind    1.524555e+07         9.432215e+09   
          solar     8.049147e+05         3.758562e+09   
Link      AC        0.000000e+00         8.395843e+08   
          h2        0.000000e+00         0.000000e+00   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         0.000000e+00   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         1.382229e-07   
          li-ion    0.000000e+00         4.810843e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             7.024836e+09      0.0           NaN  
          onwind               2.364354e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas            -8.693374e-09      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -1.826820e+05      0.0           NaN

In [7]:
generator_capital_costs = network.generators['p_nom_opt']*network.generators['capital_cost']
generator_marginal_costs = network.generators_t.p.sum()*network.generators['marginal_cost']*resolution
generator_costs = generator_capital_costs.drop('backstop') + generator_marginal_costs.drop('backstop')

generator_e = network.generators_t.p.sum().drop('backstop')*resolution

In [8]:
relevant_links = ['battery-charge', 'battery-discharge', 'h2-electrolysis', 'gas-turbine']
converter_capital_costs = network.links.loc[relevant_links]['p_nom_opt']*network.links.loc[relevant_links]['capital_cost']
converter_marginal_costs = network.links_t.p0[relevant_links].sum()*network.links.loc[relevant_links]['marginal_cost']*resolution
converter_costs = converter_capital_costs + converter_marginal_costs

In [9]:
store_capital_costs = network.stores['e_nom_opt']*network.stores['capital_cost']
store_marginal_costs = -network.links_t.p1['battery-charge'].sum()*3*network.stores.loc[['battery']]['marginal_cost']
store_marginal_costs['h2'] = 0
store_costs = store_capital_costs + store_marginal_costs

In [10]:
energy_costs = generator_costs.drop('biogas-market')
energy_costs['biogas'] = generator_costs['biogas-market'] + converter_costs['gas-turbine']

energy_costs

KeyError: "['biogas-market'] not found in axis"

In [11]:
total_costs = generator_costs.sum() + converter_costs.sum() + store_costs.sum()
total_costs

19496994434.479233

In [12]:
total_energy = network.loads_t.p.sum()*3 - network.generators_t.p['backstop'].sum()*3

In [13]:
total_costs / total_energy / 1000

Load
load    1.360407
dtype: float64

In [14]:
generator_costs / generator_e / 1000

Generator
solar     0.510024
onwind    1.176895
dtype: float64

In [ ]:
network.generators_t.p['backstop'].sum()*3/total_energy

Load
load    0.02499
dtype: float64